# Day 8

### Topics

1. Bit of a review: widgets & `ipywidgets.interact`
1. More about widgets
  * Bonus: Linking widgets
1. The NASA planet archive data
1. `bqplot` heatmap with planet data
  * Bonus: `bqplot` dashboard



First, import our usual things:

In [1]:
import ipywidgets
import numpy as np
import matplotlib.pyplot as plt
import bqplot.pyplot as bplt

## Bit of a review

Last time we got into some interactivity with `ipywidgets` and the `interact` decorator:

There are a few ways we can use this library in a few ways, the first way is through something called a "decorator" that will decorate a function.  The syntax will look a little like:

```python
@decorator
def my_function():
    # stuff
...
```

This decorator is "decorating" the function like you'd put a hat to decorate your head -- it sits on top of the function and interacts only with this function.  Let's try a simple one to start -- print out a phrase from a list.

Let's sketch out what we want to do with code statically and build from there.  First, let's make something that prints a phrase:

In [2]:
print('Hi there!')

Hi there!


Now, let's make the actual phrase an input:

In [3]:
phrase = 'I am phrase.'

print(phrase)

I am phrase.


Now, let's make this into a function:

In [4]:
def my_phrase_print(phrase):
    print(phrase)

Let's use it a few times:

In [5]:
my_phrase_print('hi!')

hi!


In [6]:
my_phrase_print('hi hi!')
my_phrase_print('helooooooo!!')

hi hi!
helooooooo!!


Now let's decorate this function with the decorator `@ipywidgets.interact`:

In [7]:
@ipywidgets.interact(phrase='')
def my_phrase_print(phrase):
    print(phrase)

interactive(children=(Text(value='', description='phrase'), Output()), _dom_classes=('widget-interact',))

Whoa!  What did we just do!  We told `ipywidgets` that our phrase was going to be text input which it figured out by the default value we put in its definition AND that the input parameter `phrase` was going to be the one we wanted to change.  Let's try another.  Also note that when we ran this cell it called the function where before we would have to call the function after we declared it.


Let's say we want to only let our user put in a selected set of phrases from a list:

In [8]:
phrases_list = ['hi there!', 'I am phrase.', 'decorators decorate!']

We could do the same thing again but with this list:

In [9]:
@ipywidgets.interact(phrase=phrases_list)
def my_phrase_print(phrase):
    print(phrase)

interactive(children=(Dropdown(description='phrase', options=('hi there!', 'I am phrase.', 'decorators decorat…

Now we have a dropdown menu instead!  There are ways to choose what kinds of widgets we get (integer sliders, dropdown menus, text boxes), but here we are having `ipywidgets` basically figure it out for us on its own.

## More about widgets

But now, lets go into ipywidgets in a bit more detail and look at specific functions and call them directly from ipywidgets.

For example, we can create a little display that increments integer numbers:

In [10]:
itext = ipywidgets.IntText()
itext

IntText(value=0)

But what if that didn't display for you?  Depending on what version of jupyter notebook you have, you might have to use "display" to actually show your widgets.  

Try (uncommenting) the following:

In [11]:
#from IPython.display import display
#display(itext)

Still didn't work?  Try refreshing and/or restarting your notebook.

*Still* not working?  You might have to install the jupyter notebook widget extension:

In [13]:
#!jupyter nbextension enable --py widgetsnbextension

If you end up running the above cell (uncommented naturally!) then you might have to refresh or restart your jupyter notebook.

Moving on: 

The value of `itext` is then stored - so we could in theory generate a toggle and then do stuff with its value:

In [14]:
itext.value

0

Note if I go up and change the toggle value I have to re-run this cell to print out the newly stored value.

I can also set the value "by hand":

In [15]:
itext.value = 10

Once I run this cell, now the toggle values are updated above.

Let's try another one, how about an integer slider:

In [29]:
slider = ipywidgets.IntSlider(min=0, max=10, step=1)
slider

IntSlider(value=0, max=10)

{'name': 'value', 'old': 5, 'new': 6, 'owner': IntSlider(value=6, max=10), 'type': 'change'}
{'name': 'value', 'old': 6, 'new': 7, 'owner': IntSlider(value=7, max=10), 'type': 'change'}
{'name': 'value', 'old': 7, 'new': 6, 'owner': IntSlider(value=6, max=10), 'type': 'change'}
{'name': 'value', 'old': 6, 'new': 5, 'owner': IntSlider(value=5, max=10), 'type': 'change'}
{'name': 'value', 'old': 5, 'new': 4, 'owner': IntSlider(value=4, max=10), 'type': 'change'}
{'name': 'value', 'old': 4, 'new': 3, 'owner': IntSlider(value=3, max=10), 'type': 'change'}


We can also check out the value of the integer slider like with the integer text:

In [35]:
slider.value

4

Ok, so the slider slides, but other than changing the value next to it nothing happens!  We want to be able to change things in our code when the widget is moved around.

This is where this call back sort of thing comes into play.  Let's look at all the things associated with this Integer Slider object:

In [32]:
dir(slider)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_cross_validation_lock',
 '_default_keys',
 '_display_callbacks',
 '_dom_classes',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_handle_custom_msg',
 '_handle_displayed',
 '_handle_msg',
 '_holding_sync',
 '_ipython_display_',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_id',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_trait',
 '_property_lock',
 '_register_validator',
 '_remove_notifiers',
 '_repr_keys',
 '_send',
 '_should_send_property

So, there is a lot of stuff here!  Let's focus on the `.observe` function:

In [33]:
slider.observe?

Ok, this has a lot of weird language about "callbacks" but the main translation of this is that this "observe" function is *observing* the slider widget object for a change and then doing something when that change happens.  Let's try associating a function to this.

We'll start by just printing out what that change is:

In [34]:
def print_slider(change):
    print(change)

Now we'll use this `.observe` function to observe for a change in the *value* of the integer slider.  Note that when we run this `.observe` we are "calling back" to our display of the integer slider above so we can actually just scroll up and it will print out what this change is.  Magic!!

In [31]:
slider.observe(print_slider, 'value')

We can also just re-display the slider as well:

In [36]:
slider

IntSlider(value=3, max=10)

{'name': 'value', 'old': 3, 'new': 4, 'owner': IntSlider(value=4, max=10), 'type': 'change'}
{'name': 'value', 'old': 4, 'new': 5, 'owner': IntSlider(value=5, max=10), 'type': 'change'}
{'name': 'value', 'old': 5, 'new': 6, 'owner': IntSlider(value=6, max=10), 'type': 'change'}
{'name': 'value', 'old': 6, 'new': 7, 'owner': IntSlider(value=7, max=10), 'type': 'change'}


Note that these two instances *of the same slider* are linked!  When one changes, so does the other one!  The print is only happening in the current "active" cell however.

So, what is this telling us, we'll we are getting a *dictionary* variable type back that gives us info of our change.  With a dictionary variable we access it like we accessed Pandas columns.  So, for example, let's re-write our function to print out the old and new values: 

In [41]:
def print_slider(change):
    print('old:', change['old'], 'new:', change['new']) # like accessing a column "old" and "new" in a Pandas dataframe

Let's make a new slider to use this function:

In [42]:
slider2 = ipywidgets.IntSlider(min=0, max=10, step=1)

And attach this new slider to this function:

In [43]:
slider2.observe(print_slider, 'value')

In [44]:
slider2

IntSlider(value=0, max=10)

old: 0 new: 1
old: 1 new: 2
old: 2 new: 1
old: 1 new: 0


Hey neat!  Ok, let's now link to widgets so when I mess with one, the other one changes.  In this sense the changes in the 2nd widget will be "driven" by the changes in the first.  You can actually link both ways but we'll just focus on the one way.

Let's make a label widget:

In [46]:
myLabel = ipywidgets.Label()
myLabel

Label(value='')

By default the label is just an empty string.  Let's set it to something:

In [47]:
myLabel.value = 'Hey Im like a string'

In [48]:
myLabel

Label(value='Hey Im like a string')

Ok, so now what we are going to do is make another slider widget and make it so that our label actually reflects the number of the slider.  

First, let's make another slider:

In [49]:
slider3 = ipywidgets.IntSlider(min=0, max=10, step=1)

Make our linking function but this time instead of printing out what old and new are, let's save it to our label's value:

In [50]:
def print_slider(change):
    #print('old:', change['old'], 'new:', change['new']) 
    myLabel.value = 'old: ' + str(change['old']) + ', new: ' + str(change['new'])

Link the slider:

In [51]:
slider3.observe(print_slider, 'value')

In [52]:
slider3

IntSlider(value=0, max=10)

In [53]:
myLabel

Label(value='old: 5, new: 4')

Ok cool, but it would probably be nice to have them both show up displayed in the same cell.  We can do this with some of the layout widgets provided in `ipywidgets`:

In [54]:
ipywidgets.HBox([slider3, myLabel]) # H for horizontal

In [55]:
ipywidgets.VBox([slider3, myLabel])

So, now we have a little bit of practice with linking widgets!  Before using them with "real" data, we are going to take a look at the data we'll be using: the NASA planet arxiv data.

### BONUS/OPTIONAL: More with widgets

Check out the notebook for the dataviz course at UIUC: https://uiuc-ischool-dataviz.github.io/spring2020/week04/prep_notebook_week04.ipynb

## The NASA planet archive data

Let's start looking at the planet dataset.  